# AP Analysis Feature Tables

This notebook assembles the requested Associated Press fields from `out/ap/analysis.db`. The three tables below correspond to the Source-, Entity-, and Version-level extracts outlined by Matt DeButts.

In [11]:
from pathlib import Path
import sqlite3

try:
    import pandas as pd
except ImportError as exc:
    raise ImportError("This notebook requires pandas. Install it with `pip install pandas`.") from exc

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 120)

db_path = Path('../out/ap/analysis.db')
assert db_path.exists(), f"Database not found at {db_path}"
print(f"Using database: {db_path}")

Using database: ../out/ap/analysis.db


## Source-level Table
Each row represents a quoted source mention. `source_location` records the character span (`start-end`) of the attribution. `first_seen`/`last_seen` originate from `sources_agg` and reflect the earliest and most recent versions containing the source.

In [12]:
source_query = """
WITH source_base AS (
    SELECT
        sm.source_id_within_article AS source_id,
        COALESCE(NULLIF(sm.source_canonical, ''), sm.source_surface) AS source_name,
        sm.char_start,
        sm.char_end,
        v.timestamp_utc AS update_time,
        sm.attributed_text AS body_of_text,
        sm.article_id AS unique_article_id,
        sm.version_id AS update_version_id,
        sa.first_seen_time AS first_seen,
        sa.last_seen_time AS last_seen,
        sm.confidence AS confidence_score
    FROM source_mentions sm
    JOIN versions v ON sm.version_id = v.version_id
    LEFT JOIN sources_agg sa
        ON sm.article_id = sa.article_id
        AND sm.news_org = sa.news_org
        AND sm.source_id_within_article = sa.source_id_within_article
)
SELECT
    source_id,
    source_name,
    printf('%d-%d', char_start, char_end) AS source_location,
    update_time,
    body_of_text,
    unique_article_id,
    update_version_id,
    first_seen,
    last_seen,
    confidence_score
FROM source_base
ORDER BY unique_article_id, update_version_id, source_id
"""

with sqlite3.connect(db_path) as conn:
    source_df = pd.read_sql_query(source_query, conn)

source_df.head()

,source_id,source_name,source_location,update_time,body_of_text,unique_article_id,update_version_id,first_seen,last_seen,confidence_score
0,s001,Barack Obama,362-425,2017-01-18 15:48:03.845167,"Manning will be freed in May, almost 30 years ahead of schedule",1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584,5.0
1,s002,Neil Eggleston,1244-1460,2017-01-18 15:48:03.845167,"""that our nation is a forgiving nation, where hard work and a commitment to rehabilitation can lead to a second chan...",1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584,5.0
2,s003,Chase Strangio,3227-3315,2017-01-18 15:48:03.845167,"""We are all better off knowing that Chelsea Manning will walk out of prison a free woman",1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584,5.0
3,s004,Paul Ryan,3553-3664,2017-01-18 15:48:03.845167,"""Chelsea Manning's treachery put American lives at risk and exposed some of our nation's most sensitive secrets",1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584,5.0
4,s005,Josh Earnest,3784-3810,2017-01-18 15:48:03.845167,"""an appropriate punishment",1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584,5.0


## Entity-level Table
`entity_location` mirrors the `start-end` character span of each recognised entity. Lifecycle timestamps are derived by grouping mentions within the same article.

In [13]:
entity_query = """
WITH entity_base AS (
    SELECT
        em.entity_id_within_article AS entity_id,
        em.entity_type,
        em.canonical_name AS entity_name,
        em.char_start,
        em.char_end,
        v.timestamp_utc AS update_time,
        em.article_id AS unique_article_id,
        em.version_id AS update_version_id
    FROM entity_mentions em
    JOIN versions v ON em.version_id = v.version_id
),
entity_bounds AS (
    SELECT
        entity_id,
        unique_article_id,
        MIN(update_time) AS first_seen,
        MAX(update_time) AS last_seen
    FROM entity_base
    GROUP BY entity_id, unique_article_id
)
SELECT
    eb.entity_id,
    eb.entity_type,
    eb.entity_name,
    printf('%d-%d', eb.char_start, eb.char_end) AS entity_location,
    eb.update_time,
    eb.unique_article_id,
    eb.update_version_id,
    b.first_seen,
    b.last_seen
FROM entity_base eb
JOIN entity_bounds b
  ON eb.entity_id = b.entity_id
 AND eb.unique_article_id = b.unique_article_id
ORDER BY eb.unique_article_id, eb.update_version_id, eb.entity_id
"""

with sqlite3.connect(db_path) as conn:
    entity_df = pd.read_sql_query(entity_query, conn)

entity_df.head()

,entity_id,entity_type,entity_name,entity_location,update_time,unique_article_id,update_version_id,first_seen,last_seen
0,e001,ORG,washington,12-30,2017-01-18 15:48:03.845167,1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584
1,e002,ORG,ap,31-33,2017-01-18 15:48:03.845167,1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584
2,e003,PERSON,barack obama,97-109,2017-01-18 15:48:03.845167,1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584
3,e004,DATE,his final days,143-157,2017-01-18 15:48:03.845167,1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584
4,e005,PERSON,chelsea manning s,220-237,2017-01-18 15:48:03.845167,1,1,2017-01-18 15:48:03.845167,2017-01-18 19:30:03.362584


## Version-level Table
One row per captured revision, with the sequential number used during ingestion.

In [14]:
version_query = """
SELECT
    v.version_id AS unique_version_id,
    v.article_id AS unique_article_id,
    v.version_num AS update_number,
    v.timestamp_utc AS timestamp
FROM versions v
ORDER BY v.article_id, v.version_num
"""

with sqlite3.connect(db_path) as conn:
    version_df = pd.read_sql_query(version_query, conn)

version_df.head()

,unique_version_id,unique_article_id,update_number,timestamp
0,1,1,0,2017-01-18 15:48:03.845167
1,11,1,1,2017-01-18 16:00:03.432431
2,112,1,2,2017-01-18 19:30:03.362584
3,2,2,0,2017-01-18 15:48:06.059903
4,88,2,1,2017-01-18 18:45:03.881484


## Extra Fields
Additional article-level metadata is available for analysts who need to enrich the requested tables:
- `news_org`: newsroom label supplied by the AP feed.
- `url`: canonical story URL collected during ingestion.
- `title_first` / `title_final`: earliest and latest headlines observed.
- `original_publication_time`: UTC timestamp of the seed version.
- `total_edits`: number of revisions after the initial capture.
- `is_live_blog`: flag indicating rolling live coverage detected by the pipeline.

In [15]:
extra_query = """
SELECT
    article_id AS unique_article_id,
    news_org,
    url,
    title_first,
    title_final,
    original_publication_time,
    total_edits,
    is_live_blog
FROM articles
ORDER BY article_id
"""

with sqlite3.connect(db_path) as conn:
    extra_df = pd.read_sql_query(extra_query, conn)

extra_df.head()

,unique_article_id,news_org,url,title_first,title_final,original_publication_time,total_edits,is_live_blog
0,1,ap,http://hosted.ap.org/dynamic/stories/U/US_OBAMA_CLEMENCY?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT,News from The Associated Press,News from The Associated Press,2017-01-18 15:48:03.845167,2,0
1,2,ap,http://hosted.ap.org/dynamic/stories/U/US_TRUMP_INAUGURATION_SPEECH?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT,News from The Associated Press,News from The Associated Press,2017-01-18 15:48:06.059903,3,0
2,5,ap,http://hosted.ap.org/dynamic/stories/U/US_TRUMP_HEALTH_SECRETARY?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT,News from The Associated Press,News from The Associated Press,2017-01-18 15:48:13.285719,17,0
3,10,ap,http://hosted.ap.org/dynamic/stories/B/BBO_HALL_OF_FAME?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT,News from The Associated Press,News from The Associated Press,2017-01-18 15:48:27.663459,4,0
4,11,ap,http://hosted.ap.org/dynamic/stories/U/US_SCI_HOTTEST_YEAR?SITE=AP&SECTION=HOME&TEMPLATE=DEFAULT,News from The Associated Press,News from The Associated Press,2017-01-18 16:00:19.054547,2,0


### source-extras
Additional context captured for each source mention:
- `source_type`: sector label (e.g., official, civilian) assigned by the source prompt.
- `speech_style`: framing of the attribution (direct quote, indirect, paraphrase).
- `attribution_verb`: verb used in the attribution clause.
- `is_in_title` / `is_in_lede`: flags marking whether the source appears in the headline or opening graf.
- `is_anonymous`: boolean flag for unnamed sources.
- `anonymous_description` / `anonymous_domain`: pipeline-provided description and sector for anonymous voices.
- `evidence_type` / `evidence_text`: what evidence the source supplies and the supporting snippet.
- `narrative_function` / `centrality`: narrative role label and relative prominence emitted by the narrative prompt.
- `perspective`: JSON-encoded list describing whose perspective the source represents.
- `doubted`: whether the article signals skepticism toward the source.
- `hedge_count`: number of hedging cues found near the attribution.
- `hedge_markers` / `epistemic_verbs`: JSON-encoded lists of hedge tokens and epistemic verbs detected.
- `hedge_stance` / `hedge_confidence`: model judgement of hedge orientation and confidence (1–5 scale).
- `prominence_lead_pct`: normalised location of the mention within the processed text.

In [16]:
source_extras_query = """
SELECT
    sm.source_id_within_article AS source_id,
    sm.article_id AS unique_article_id,
    sm.version_id AS update_version_id,
    sm.source_type,
    sm.speech_style,
    sm.attribution_verb,
    sm.is_in_title,
    sm.is_in_lede,
    sm.is_anonymous,
    sm.anonymous_description,
    sm.anonymous_domain,
    sm.evidence_type,
    sm.evidence_text,
    sm.narrative_function,
    sm.centrality,
    sm.perspective,
    sm.doubted,
    sm.hedge_count,
    sm.hedge_markers,
    sm.epistemic_verbs,
    sm.hedge_stance,
    sm.hedge_confidence,
    sm.prominence_lead_pct
FROM source_mentions sm
ORDER BY sm.article_id, sm.version_id, sm.source_id_within_article
"""

with sqlite3.connect(db_path) as conn:
    source_extras_df = pd.read_sql_query(source_extras_query, conn)

source_extras_df.head()

,source_id,unique_article_id,update_version_id,source_type,speech_style,attribution_verb,is_in_title,is_in_lede,is_anonymous,anonymous_description,anonymous_domain,evidence_type,evidence_text,narrative_function,centrality,perspective,doubted,hedge_count,hedge_markers,epistemic_verbs,hedge_stance,hedge_confidence,prominence_lead_pct
0,s001,1,1,government,indirect,announced,0,0,0,,unknown,official_statement,"Manning will be freed in May, almost 30 years ahead of schedule","""Key Actor"": The source is the main actor in the news article, making key decisions that drive the story forward.",High,"[\n ""Authoritative""\n]",0,0,[],[],neutral,5.0,0.051297
1,s002,1,1,government,direct,said,0,0,0,,unknown,official_statement,"""that our nation is a forgiving nation, where hard work and a commitment to rehabilitation can lead to a second chan...","""Expert Context"": This source provides context and explanation for the actions taken by the main actor, offering ins...",Medium,"[\n ""Informative"",\n ""Supportive""\n]",0,0,[],[],neutral,5.0,0.176279
2,s003,1,1,civil_society,direct,said,0,0,0,,unknown,eyewitness,"""We are all better off knowing that Chelsea Manning will walk out of prison a free woman""","""Advocate"": The source is an advocate for the individual affected by the main actor's decision, providing a personal...",Medium,"[\n ""Supportive""\n]",0,0,[],[],supportive,5.0,0.457276
3,s004,1,1,government,direct,said,0,0,0,,unknown,official_statement,"""Chelsea Manning's treachery put American lives at risk and exposed some of our nation's most sensitive secrets""","""Counterpoint"": This source provides an opposing view to the main actor's decision, highlighting potential drawbacks...",Medium,"[\n ""Against"",\n ""Skeptical""\n]",1,0,[],[],skeptical,5.0,0.503472
4,s005,1,1,government,direct,said,0,0,0,,unknown,official_statement,"""an appropriate punishment""","""Spokesperson"": The source serves as a spokesperson for the main actor, providing additional information and clarifi...",Low,"[\n ""Informative"",\n ""Neutral""\n]",0,0,[],[],neutral,5.0,0.536205


### version-extras
Per-version analytics and pairwise revision deltas:
- `distinct_sources`: count of unique canonical sources in the version.
- `institutional_share_words`: share of attributed words spoken by institutions.
- `anonymous_source_share_words`: share of attributed words from anonymous speakers.
- `hedge_density_per_1k`: hedges per 1,000 tokens in the version.
- `delta_minutes`: minutes elapsed between consecutive versions in a pair.
- `tokens_added` / `tokens_deleted`: token-level addition and deletion counts across the pair.
- `percent_text_new`: proportion of the later version that is newly written text.
- `movement_confidence`: confidence score for the movement summary prompt.
- `edit_type` / `edit_confidence`: classification of the edit (A3) and its confidence.
- `angle_changed` / `angle_confidence`: whether the framing angle shifted and model confidence.
- `title_jaccard_prev` / `title_jaccard_curr`: Jaccard similarity of headline/lede excerpts between versions.
- `summary_jaccard`: Jaccard similarity for the body summary.

In [17]:
version_metrics_query = """
SELECT
    vm.version_id,
    vm.article_id AS unique_article_id,
    vm.distinct_sources,
    vm.institutional_share_words,
    vm.anonymous_source_share_words,
    vm.hedge_density_per_1k
FROM version_metrics vm
ORDER BY vm.article_id, vm.version_id
"""

with sqlite3.connect(db_path) as conn:
    version_metrics_df = pd.read_sql_query(version_metrics_query, conn)

version_metrics_df.head()

,version_id,unique_article_id,distinct_sources,institutional_share_words,anonymous_source_share_words,hedge_density_per_1k
0,1,1,6,0.845455,0.209091,0.762777
1,11,1,7,0.756098,0.186992,0.000000
2,112,1,7,0.855932,0.194915,3.252033
3,170,2,6,0.000000,0.000000,0.000000
4,2,2,4,0.000000,0.000000,5.434783


In [18]:
version_pairs_query = """
SELECT
    vp.article_id AS unique_article_id,
    vp.from_version_id,
    vp.to_version_id,
    vp.from_version_num,
    vp.to_version_num,
    vp.delta_minutes,
    vp.tokens_added,
    vp.tokens_deleted,
    vp.percent_text_new,
    vp.movement_confidence,
    vp.edit_type,
    vp.edit_confidence,
    vp.angle_changed,
    vp.angle_confidence,
    vp.title_jaccard_prev,
    vp.title_jaccard_curr,
    vp.summary_jaccard
FROM version_pairs vp
ORDER BY vp.article_id, vp.from_version_num
"""

with sqlite3.connect(db_path) as conn:
    version_pairs_df = pd.read_sql_query(version_pairs_query, conn)

version_pairs_df.head()

,unique_article_id,from_version_id,to_version_id,from_version_num,to_version_num,delta_minutes,tokens_added,tokens_deleted,percent_text_new,movement_confidence,edit_type,edit_confidence,angle_changed,angle_confidence,title_jaccard_prev,title_jaccard_curr,summary_jaccard
0,1,1,112,0,2,221.991957,71,142,0.071214,1,content_update,5.0,0,5,0.000000,0.000000,0.862568
1,2,2,653,0,3,3101.961231,1171,543,0.792823,1,content_update,5.0,0,5,0.038462,0.037037,0.138173
2,5,5,53399,0,17,408261.473382,58,828,0.537037,1,content_update,5.0,0,5,0.023810,0.000000,0.082828
3,10,10,493,0,4,2156.838386,460,540,0.539906,1,content_update,5.0,0,5,0.038462,0.000000,0.254111
4,11,12,118,0,2,224.941133,546,25,0.833588,1,content_update,5.0,0,5,0.050000,0.029412,0.238235


### entity-extras
No additional entity-level analytics are stored beyond the core fields above; this section remains available for future features such as entity sentiment or co-reference tracking.